In [1]:
import pandas as pd
import numpy as np
from datasets import Dataset, Features, load_metric

from PIL import Image
import torch
from torchvision import transforms
from transformers import ViTForImageClassification, ViTFeatureExtractor, Trainer, TrainingArguments, ViTImageProcessor

from src import dataloader as ds


In [2]:
torch.zeros(1).cuda()

tensor([0.], device='cuda:0')

In [3]:
print(torch.cuda.is_available())

True


In [4]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

In [5]:
file_path = 'fer2013.tar.gz'
data_path = ds.unpack_tar_gz(file_path)

Target folder 'data' is not empty. Assuming the file is already unpacked.


In [6]:
df = pd.read_csv(data_path)

In [7]:
def process_for_huggingface_dataset(data):
    image_list = []
    image_labels = data['emotion'].astype(int).tolist()
    
    for pixels in data['pixels']:
        # Convert pixel string to an array of integers
        image = np.fromstring(pixels, dtype=int, sep=' ')
        # Reshape to 48x48
        image = image.reshape((48, 48))
        # Convert grayscale to RGB by repeating the grayscale values across three channels
        image_rgb = np.stack([image] * 3, axis=-1)
        # Convert to list format
        image_list.append(image_rgb.tolist())

    # Create a DataFrame with images and labels
    output_df = pd.DataFrame({'img': image_list, 'label': image_labels})
    
    return output_df

In [8]:
# this will take a while
train_hf = process_for_huggingface_dataset(df[df['Usage'] == 'Training'])
val_hf = process_for_huggingface_dataset(df[df['Usage'] == 'PublicTest'])
test_hf = process_for_huggingface_dataset(df[df['Usage'] == 'PrivateTest'])

In [9]:
train_hf.head()

,img,label
0,"[[[70, 70, 70], [80, 80, 80], [82, 82, 82], [7...",0
1,"[[[151, 151, 151], [150, 150, 150], [147, 147,...",0
2,"[[[231, 231, 231], [212, 212, 212], [156, 156,...",2
3,"[[[24, 24, 24], [32, 32, 32], [36, 36, 36], [3...",4
4,"[[[4, 4, 4], [0, 0, 0], [0, 0, 0], [0, 0, 0], ...",6


In [10]:
train_dataset = Dataset.from_pandas(train_hf)
val_dataset = Dataset.from_pandas(val_hf)
test_dataset = Dataset.from_pandas(test_hf)

In [11]:
train_dataset

Dataset({
    features: ['img', 'label'],
    num_rows: 28709
})

In [12]:
num_classes = len(set(train_hf['label']))
labels = train_dataset.features['label']
num_classes, labels

(7, Value(dtype='int64', id=None))

In [13]:
train_dataset[0]['img']

[[[70, 70, 70],
  [80, 80, 80],
  [82, 82, 82],
  [72, 72, 72],
  [58, 58, 58],
  [58, 58, 58],
  [60, 60, 60],
  [63, 63, 63],
  [54, 54, 54],
  [58, 58, 58],
  [60, 60, 60],
  [48, 48, 48],
  [89, 89, 89],
  [115, 115, 115],
  [121, 121, 121],
  [119, 119, 119],
  [115, 115, 115],
  [110, 110, 110],
  [98, 98, 98],
  [91, 91, 91],
  [84, 84, 84],
  [84, 84, 84],
  [90, 90, 90],
  [99, 99, 99],
  [110, 110, 110],
  [126, 126, 126],
  [143, 143, 143],
  [153, 153, 153],
  [158, 158, 158],
  [171, 171, 171],
  [169, 169, 169],
  [172, 172, 172],
  [169, 169, 169],
  [165, 165, 165],
  [129, 129, 129],
  [110, 110, 110],
  [113, 113, 113],
  [107, 107, 107],
  [95, 95, 95],
  [79, 79, 79],
  [66, 66, 66],
  [62, 62, 62],
  [56, 56, 56],
  [57, 57, 57],
  [61, 61, 61],
  [52, 52, 52],
  [43, 43, 43],
  [41, 41, 41]],
 [[65, 65, 65],
  [61, 61, 61],
  [58, 58, 58],
  [57, 57, 57],
  [56, 56, 56],
  [69, 69, 69],
  [75, 75, 75],
  [70, 70, 70],
  [65, 65, 65],
  [56, 56, 56],
  [54, 54, 54]

In [14]:
model_id = 'google/vit-base-patch16-224'
processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224')
model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224')
model.to(device)

ViTForImageClassification(
  (vit): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTSdpaAttention(
            (attention): ViTSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_fe

In [15]:
# Preprocess function
def preprocess_function(examples):
    processed_images = []
    for image in examples['img']:
        # Convert to PIL Image
        image = np.array(image, dtype=np.uint8)
        image = Image.fromarray(image)
        # Resize to 224x224
        image = image.resize((224, 224))
        # Ensure it's RGB
        image = image.convert("RGB")
        processed_images.append(np.array(image))
    
    inputs = processor(images=processed_images, return_tensors="pt")
    inputs['labels'] = examples['label']
    return inputs

In [16]:
train_dataset = train_dataset.map(preprocess_function, batched=True, remove_columns=["img"])
val_dataset = val_dataset.map(preprocess_function, batched=True, remove_columns=["img"])
test_dataset = test_dataset.map(preprocess_function, batched=True, remove_columns=["img"])


Map:   0%|          | 0/28709 [00:00<?, ? examples/s]

Map:   0%|          | 0/3589 [00:00<?, ? examples/s]

Map:   0%|          | 0/3589 [00:00<?, ? examples/s]

In [17]:
train_dataset

Dataset({
    features: ['label', 'pixel_values', 'labels'],
    num_rows: 28709
})

In [18]:
# Define the metrics
accuracy_metric = load_metric("accuracy")

def compute_metrics(p):
    return accuracy_metric.compute(predictions=np.argmax(p.predictions, axis=1), references=p.label_ids)


C:\Users\Tommy\AppData\Local\Temp\ipykernel_30968\1390381944.py:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  accuracy_metric = load_metric("accuracy")
C:\Users\Tommy\PycharmProjects\ComputerVision\.venv\lib\site-packages\datasets\load.py:759: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [20]:
# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-4,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=5,
    weight_decay=0.01
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

# Train the model
trainer.train()

C:\Users\Tommy\PycharmProjects\ComputerVision\.venv\lib\site-packages\transformers\training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\Tommy\PycharmProjects\ComputerVision\.venv\lib\site-packages\transformers\models\vit\modeling_vit.py:253: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  context_layer = torch.nn.functional.scaled_dot_product_attention(


Epoch,Training Loss,Validation Loss,Accuracy
1,1.229500,0.970429,0.643912
2,0.798400,0.901578,0.665924
3,0.523200,0.966025,0.684035
4,0.223100,1.215841,0.702703
5,0.088700,1.472014,0.700195


TrainOutput(global_step=4490, training_loss=0.5411254122421842, metrics={'train_runtime': 17345.0595, 'train_samples_per_second': 8.276, 'train_steps_per_second': 0.259, 'total_flos': 1.1223085452511887e+19, 'train_loss': 0.5411254122421842, 'epoch': 5.0})

In [21]:
# Evaluate the model
trainer.evaluate()

{'eval_loss': 1.4720141887664795,
 'eval_accuracy': 0.7001950404012259,
 'eval_runtime': 338.168,
 'eval_samples_per_second': 10.613,
 'eval_steps_per_second': 0.334,
 'epoch': 5.0}

In [23]:
trainer.save_model("fine_tuned_vit")